In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns
from sklearn.datasets import make_regression
import pandas as pd
import pymc as pm

## Time Series

In [4]:
def seasonal(amplitude, periodicity, timestamp):
    return amplitude * np.sin((2*np.pi)/periodicity*timestamp)

In [5]:
def gen_dem(x):
    demand = 20*x + seasonal(50, 5, x) + np.random.normal(0, 30) + seasonal(120, 20, x)
    return demand

In [6]:
t = np.arange(0,1,0.001)
y = np.array([gen_dem(i) for i in np.arange(0,100,0.1)])
y_stand = y/y.max()

In [7]:
with pm.Model() as linear:
    alpha = pm.Normal("alpha", mu=0, sigma=5)
    beta = pm.Normal("beta", mu=0, sigma=5)
    sigma = pm.HalfNormal("sigma", sigma=5)

    trend = pm.Deterministic("trend", alpha + beta * t)
    pm.Normal("likelihood", mu=trend, sigma=sigma, observed=y_stand)

    linear_prior = pm.sample_prior_predictive()

## Regression

In [2]:
obs = make_regression(100, n_features=1, n_targets=1, noise=4, bias=4)
x_obs = obs[0]
y_obs = obs[1]

In [ ]:
with pm.Model() as lr:

    # not observed -> prior distribution
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=1)

    mu = alpha + beta * x_obs

    # observed -> likelihood distribution
    y = pm.Normal("y", mu=mu, sigma=sigma, observed=y_obs)

A/B Testing

In [8]:
df = pd.read_csv("C:\\Data\\WA_Marketing-Campaign.csv")

In [ ]:
with pm.Model() as m:


    # mu_hyper = pm.Normal("mu_hyper", mu=0, sigma=1)
    sigma_hyper = pm.HalfNormal("sigma_hyper", sigma=2)
    mu = pm.Normal("mu", mu=30, sigma=1)
    # mu = pm.Normal("mu", mu=mu_hyper, sigma=sigma_hyper, shape=df["MarketSize"].nunique())

    y = pm.Normal("y", mu=mu, sigma=sigma_hyper, observed=df.SalesInThousands)

    idata = pm.sample(5000)